In [2]:
!pip install python-dotenv pymongo python-telegram-bot google-generativeai nest_asyncio
!pip install --upgrade httpx
!pip install python-telegram-bot --upgrade
!pip install google-cloud-vision
!pip install vaderSentiment
!pip install --upgrade python-telegram-bot
!pip install deep-translator


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 31.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 669.5/669.5 kB 38.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.6/313.6 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 514.6/514.6 kB 9.5 MB/s eta 0:00:00


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 2.9 MB/s eta 0:00:00


In [ ]:
import os
import asyncio
from dotenv import load_dotenv
from pymongo import MongoClient, errors
from telegram import Update, ReplyKeyboardMarkup, KeyboardButton
from telegram.ext import ApplicationBuilder, CommandHandler, MessageHandler, filters, CallbackContext
import google.generativeai as genai
import requests
from io import BytesIO
from telegram.error import TelegramError
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from deep_translator import GoogleTranslator
import random

# Load environment variables
load_dotenv()
MONGO_URI = os.getenv("MONGO_URI")
BOT_TOKEN = os.getenv("TELEGRAM_BOT_TOKEN")
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")

# MongoDB Setup with Error Handling
try:
    client = MongoClient(MONGO_URI)
    db = client["telegram_bot"]
    users_collection = db["users"]
    users_collection.create_index("chat_id", unique=True)  # Index for faster querying
    analytics_collection = db["analytics"]
except errors.ConnectionFailure:
    print("❌ Failed to connect to MongoDB. Check your connection string.")

# Initialize Telegram Bot
app = ApplicationBuilder().token(BOT_TOKEN).build()

# Configure Gemini API
genai.configure(api_key=GEMINI_API_KEY)

# Sentiment Analysis Setup
analyzer = SentimentIntensityAnalyzer()
from deep_translator import GoogleTranslator
from telegram import Update
from telegram.ext import CommandHandler, CallbackContext

# Translation function
def translate_text(text, target_language="en"):
    try:
        translation = GoogleTranslator(source='auto', target=target_language).translate(text)
        return translation
    except Exception as e:
        print(f"Error in translation: {e}")
        return None

# Command to translate text
async def translate_command(update: Update, context: CallbackContext):
    user_input = " ".join(context.args)  # Get the text after the command

    if not user_input:
        await update.message.reply_text("⚠️ Please provide some text to translate. Example: `/translate Hello`")
        return

    target_language = "en"  # Default to English
    if len(context.args) > 1:
        target_language = context.args[0]  # First argument as target language code

    translated_text = translate_text(user_input, target_language)
    if translated_text:
        await update.message.reply_text(f"🔤 **Translated Text:**\n{translated_text}")
    else:
        await update.message.reply_text("⚠️ Error in translation. Please try again later.")

# Add handler for /translate command
app.add_handler(CommandHandler("translate", translate_command))


# Start Command - User Registration
async def start(update: Update, context: CallbackContext):
    user = update.effective_user
    try:
        if not users_collection.find_one({"chat_id": user.id}):
            users_collection.insert_one({
                "chat_id": user.id,
                "first_name": user.first_name,
                "username": user.username
            })
            await update.message.reply_text(f"✅ Welcome {user.first_name}! You have been successfully registered.")
        else:
            await update.message.reply_text("✅ You're already registered!")
    except errors.PyMongoError as e:
        await update.message.reply_text("⚠️ Error saving your details. Please try again later.")
        print(e)

app.add_handler(CommandHandler("start", start))

# Help Command
async def help_command(update: Update, context: CallbackContext):
    help_text = (
        "🤖 **Available Commands:**\n"
        "/start - Register with the bot\n"
        "/phone - Share your phone number\n"
        "/websearch <query> - Perform a web search\n"
        "/settings - Customize bot settings\n"
        "/translate- Translate the text{/translate fr Hello}\n"
        "/funfact - Get a random fun fact\n"
        "//motivationalquote - Get a random quote\n"
        "💬 Send text or images to chat with AI."
    )
    await update.message.reply_text(help_text)

app.add_handler(CommandHandler("help", help_command))

# Request Phone Number
async def request_phone(update: Update, context: CallbackContext):
    keyboard = [[KeyboardButton("📞 Share Phone", request_contact=True)]]
    reply_markup = ReplyKeyboardMarkup(keyboard, one_time_keyboard=True)
    await update.message.reply_text("📲 Please share your phone number:", reply_markup=reply_markup)

# Save Phone Number
async def save_phone(update: Update, context: CallbackContext):
    contact = update.message.contact
    try:
        users_collection.update_one(
            {"chat_id": update.effective_user.id},
            {"$set": {"phone": contact.phone_number}},
            upsert=True
        )
        await update.message.reply_text("✅ Phone number saved!")
    except errors.PyMongoError as e:
        await update.message.reply_text("⚠️ Could not save your phone number. Try again later.")
        print(e)

app.add_handler(CommandHandler("phone", request_phone))
app.add_handler(MessageHandler(filters.CONTACT, save_phone))

# AI Chat with Gemini API
async def chat(update: Update, context: CallbackContext):
    user_input = update.message.text
    try:
        model = genai.GenerativeModel("gemini-pro")
        response = model.generate_content(user_input)
        bot_response = response.text if response else "⚠️ Sorry, I couldn't understand that."

        # Sentiment Analysis
        sentiment = analyze_sentiment(user_input)
        bot_response += f"\n\nSentiment: {sentiment}"

        await update.message.reply_text(bot_response)
    except Exception as e:
        await update.message.reply_text("⚠️ Error processing your message. Please try again.")
        print(e)

def analyze_sentiment(text):
    score = analyzer.polarity_scores(text)
    if score['compound'] >= 0.05:
        return "Positive"
    elif score['compound'] <= -0.05:
        return "Negative"
    else:
        return "Neutral"

app.add_handler(MessageHandler(filters.TEXT & ~filters.COMMAND, chat))

async def handle_image(update: Update, context: CallbackContext):
    try:
        photo = update.message.photo[-1]
        file = await photo.get_file()
        file_url = file.file_path
        response = requests.get(file_url)
        image_bytes = BytesIO(response.content)
        if not image_bytes.getvalue():
            await update.message.reply_text("⚠️ The image seems empty or invalid.")
            return
        model = genai.GenerativeModel("gemini-1.5-flash")
        analysis = model.generate_content([image_bytes])
        response_text = analysis.text if hasattr(analysis, 'text') else "⚠️ No response from AI."
        await update.message.reply_text(f"📷 **Image Analysis:**\n{response_text}")
    except Exception as e:
        print(e)
        await update.message.reply_text("⚠️ Error analyzing the image.")
app.add_handler(MessageHandler(filters.PHOTO, handle_image))



# Web Search using Gemini API
async def web_search(update: Update, context: CallbackContext):
    query = " ".join(context.args)
    if not query:
        await update.message.reply_text("⚠️ Please enter a search query! Example: `/websearch AI news`")
        return

    try:
        model = genai.GenerativeModel("gemini-pro")
        response = model.generate_content(f"Search the web for: {query}")
        search_results = response.text if response else "⚠️ No search results found."

        await update.message.reply_text(f"🔍 **Search Results for '{query}':**\n{search_results}")

    except Exception as e:
        await update.message.reply_text("⚠️ Error performing web search. Try again later.")
        print(e)

app.add_handler(CommandHandler("websearch", web_search))

# Fun Fact/Quote Commands


# Fun Fact Command using Gemini API with Timeout
async def funfact_command(update: Update, context: CallbackContext):
    try:
        # Set a timeout for the request
        timeout = 10  # seconds

        # Create a task to generate a fun fact asynchronously
        async def fetch_fun_fact():
            model = genai.GenerativeModel("gemini-pro")
            response = model.generate_content("Generate a random fun fact.")
            return response

        # Use asyncio to fetch the fun fact with a timeout
        task = asyncio.create_task(fetch_fun_fact())
        response = await asyncio.wait_for(task, timeout=timeout)

        # Check if the response contains text and handle it
        if response and hasattr(response, 'text'):
            fun_fact = response.text
        else:
            fun_fact = "⚠️ Couldn't generate a fun fact at the moment."

        # Send the generated fun fact to the user
        await update.message.reply_text(f"🧐 Fun Fact: {fun_fact}")

    except asyncio.TimeoutError:
        await update.message.reply_text("⚠️ The request took too long to process. Please try again later.")
        print("Timeout Error: The request took too long.")
    except Exception as e:
        await update.message.reply_text("⚠️ Error generating the fun fact. Try again later.")
        print("Gemini API Error:", e)
app.add_handler(CommandHandler("funfact",funfact_command))


# Motivational Quote Command
async def quote_command(update: Update, context: CallbackContext):
    try:
        # Request a motivational quote from the Gemini API
        model = genai.GenerativeModel("gemini-pro")
        response = model.generate_content("Generate a motivational quote for today.")

        # Get the generated quote text
        motivational_quote = response.text if response else "⚠️ Couldn't generate a quote at the moment."

        await update.message.reply_text(f"💪 **Motivational Quote:**\n{motivational_quote}")
    except Exception as e:
        await update.message.reply_text("⚠️ Error generating the quote. Try again later.")
        print("Gemini API Error:", e)

app.add_handler(CommandHandler("motivationalquote", quote_command))


# /settings Command
async def settings_command(update: Update, context: CallbackContext):
    await update.message.reply_text("⚙️ Settings feature coming soon!")

app.add_handler(CommandHandler("settings", settings_command))

# Referral System
async def referral(update: Update, context: CallbackContext):
    referrer = update.effective_user.id
    referral_code = "REF" + str(referrer)  # Generate unique referral code
    await update.message.reply_text(f"Your referral code: {referral_code}\nShare this code with your friends!")

app.add_handler(CommandHandler("referral", referral))

# Run the bot with polling
print("🚀 Bot is running...")
import nest_asyncio
nest_asyncio.apply()

app.run_polling()


🚀 Bot is running...
